## Create GPlates GPML VGP file from pole list

- Copyright (C) 2015 Michael G. Tetley
- University of Sydney / California Institute of Technology
- Contact: michael.tetley@sydney.edu.au

Script takes input paleomagnetic poles in CSV format with the following headers and creates GPlates GPML VGP input file.


<b>Q | A95 | Formation | SLat | SLon | PLat | Plon | AgeNominal | AgeLower | AgeUpper</b>

<b>Q</b> = Van der Voo quality criteria (0 - 7) [*optional*]<br>
<b>A95</b> = 95% confidence limit on pole in degrees (e.g. 6.2)<br>
<b>Formation</b> = Geological formation sampled (e.g. Snake River Plain)<br>
<b>SLat</b> = Sample site decimal latitude (e.g. -56.2)<br>
<b>SLon</b> = Sample site decimal longitude (e.g. 134.8)<br>
<b>PLat</b> = Palaeomagnetic pole decimal latitude (e.g. -56.2)<br>
<b>PLon</b> = Palaeomagnetic pole decimal longitude (e.g. 134.8)<br>
<b>AgeNominal</b> = Mean / representative age of sampled formation (e.g. 150.5)<br>
<b>AgeLower</b> = Lower bound on age uncertainty (e.g. 145.2)<br>
<b>AgeUpper</b> = Upper bound on age uncertainty (e.g. 156.9)<br>

### Required libraries

In [1]:
import numpy as np
import pygplates as pgp
import pandas as pd
import os

In [2]:
ls

converting_sam_to_magic.md  images/
create_vgp.ipynb            make_pole_gpml/
demag_gui_doc.md            making_gpml_pole_lists.md
demag_gui_doc.pdf           making_sam_files.md


### Import data

In [13]:
# Load pole data
pmag_file = './make_pole_gpml/Mongolia_Pole_List.csv'
pmag_data = pd.read_csv(pmag_file)

# Load polygon file (generally coastlines)
polyFile = "./make_pole_gpml/Domeier2014_land.gpml"

In [14]:
# Iterate through palaeomagnetic pole dataset (pmag_data) and build lists of unique terranes and continental blocks
terrane_list = []
plateid_list = []

for i, terrane in enumerate(pmag_data.Terrane.values):
        
    if terrane not in terrane_list:
        
        terrane_list.append(pmag_data.Terrane[i])
        
        
for j, plateid in enumerate(pmag_data.Plate_ID.values):
    
    if np.isnan(plateid):
        
        plateid_list.append(666)
        
    else:
        
        plateid_list.append(pmag_data.Plate_ID[j])
        
        
#######

# Iterate through palaeomagnetic pole data set and build Pandas DataFrame of required data
pole_list = []

for terrane in terrane_list:
    
    for k, item in enumerate(pmag_data.Terrane.values):
        
        if item == terrane:
            
            # Create dictionary for DataFrame
            pole_list.append({'Terrane': pmag_data.Terrane[k], 'Formation': pmag_data.Formation[k],  
                              'PLat': pmag_data.PLat[k], 'PLon': pmag_data.PLon[k],
                              'SLat': pmag_data.SLat[k],'SLon': pmag_data.SLon[k], 'PlateID': plateid_list[k],
                              'AgeNominal': pmag_data.AgeNominal[k], 'AgeLower': pmag_data.AgeLower[k], 
                              'AgeUpper': pmag_data.AgeUpper[k], 'A95': pmag_data.A95[k],
                              'Q': pmag_data.Q[k]})
            
            
# Create and display Pandas DataFrame   
raw_polesDF = pd.DataFrame(pole_list)
raw_polesDF

,A95,AgeLower,AgeNominal,AgeUpper,Formation,PLat,PLon,PlateID,Q,SLat,SLon,Terrane
0,3.6,0.0,0.5,1.0,West Eifel volcanics,-80.6,267.5,302,4,50.2,6.70,Stable Europe
1,4.4,0.0,0.5,1.0,East Eifel volcanics,-86.4,296.1,302,4,50.4,7.30,Stable Europe
2,12.9,6.0,8.0,10.0,Volcanics NW Germany,-84.3,357.7,302,3,48.0,9.00,Stable Europe
3,1.8,8.0,9.5,11.0,"Prado section, Teruel, Spain",-78.9,328.3,302,5,40.3,358.80,Stable Europe
4,3.5,9.0,10.0,11.0,"Cascante, Spain",-77.4,314.2,302,5,40.2,358.90,Stable Europe
5,6.9,9.0,11.5,14.0,Velay Oriental volcanics,-84.1,351.2,302,5,45.0,4.20,Stable Europe
6,1.5,10.7,12.0,12.8,"Orera, Spain",-72.4,352.0,302,5,41.3,358.50,Stable Europe
7,4.4,18.0,24.0,30.0,Volcanics Germany,-77.8,310.8,302,4,50.8,8.00,Stable Europe
8,3.4,23.0,34.0,45.0,Hocheifel volcanics,-80.8,2.0,302,3,50.3,7.00,Stable Europe
9,1.5,45.0,49.5,54.0,"Lundy island dikes, Wales",-83.0,335.0,302,5,51.2,355.30,Stable Europe


### Perform 'point in polygon' test to assign plate ID to pole based on sampling location

In [15]:
featureCollection = pgp.FeatureCollectionFileFormatRegistry()
featureSet = featureCollection.read(polyFile)
 
try:
    featureSet = featureCollection.read(polyFile)

except pgp.OpenFileForReadingError:
    print "Doesn't work like that silly."


plateIDs = []
plates = []


# Array to contain correct line numbers from full dataset to make sure the correct values are assiciated the correct poles.
line = []
count = 0
polyCount = 0

for i in xrange(0, len(raw_polesDF)):
    
    count = count + 1
    
    if pd.isnull(raw_polesDF.SLat[i]) == False:
        inputlatLon = pgp.LatLonPoint(raw_polesDF.SLat[i], raw_polesDF.SLon[i])
        latLonPoint = pgp.convert_lat_lon_point_to_point_on_sphere(inputlatLon)
              
    for feature in featureSet:
        
        for geometry in feature.get_geometries():
            
            polygon = pgp.PolygonOnSphere(geometry)
            isPoly = polygon.is_point_in_polygon(latLonPoint)

            if isPoly == True:

                if feature.get_reconstruction_plate_id() != 346:

                    #plateIDs.append(feature.get_reconstruction_plate_id())
                    plates.append(feature.get_name())
                    line.append(i)

                # Uncomment this line for verbose output
                # print "Count: " + str(count) + ", Line: " + str(line[polyCount]) + ", Value: " + str(raw_polesDF.a95[i]) + \
                # ", PlateID: " + str(plateIDs[polyCount]) + ", Plate: " + str(plates[polyCount])

                polyCount = polyCount + 1

print "Polygon test complete. Located and assigned plate ID's to", len(plates), "poles."

Polygon test complete. Located and assigned plate ID's to 169 poles.


### Create and populate FeatureCollection

In [16]:
# Loop through list of poles and produce GPML file of multiple VGPs.
poleLat = []
poleLon = []
poleName = []
poleInc = []
poleDec = []
poleSiteLat = []
poleSiteLon = []
poleNominalAge = []
poleA95 = []
poleAgeLowerLimit = []
poleAgeUpperLimit = []
plateID = []
line = []


count = 0

for i in xrange(0, len(raw_polesDF)):
    
    line.append(i)
    
    if np.isfinite(raw_polesDF.SLat[line[i]]) and np.isfinite(raw_polesDF.SLon[line[i]]) and \
       np.isfinite(raw_polesDF.AgeLower[line[i]]) and np.isfinite(raw_polesDF.AgeUpper[line[i]]):

        poleLat.append(raw_polesDF.PLat[line[i]])
        poleLon.append(raw_polesDF.PLon[line[i]])
    
        poleName.append(raw_polesDF.Formation[line[i]])
        poleSiteLat.append(raw_polesDF.SLat[line[i]])
        poleSiteLon.append(raw_polesDF.SLon[line[i]])
        poleNominalAge.append(raw_polesDF.AgeNominal[line[i]])
        poleA95.append(raw_polesDF.A95[line[i]])
        
        poleAgeLowerLimit.append(raw_polesDF.AgeLower[line[i]])
        poleAgeUpperLimit.append(raw_polesDF.AgeUpper[line[i]])
        
        plateID.append(raw_polesDF.PlateID[line[i]])
        #plateID.append(000)

        count = count + 1
        
    # Print if any of the isinf tests fail
    else:
        
        print "DataFrame #", line[i], ': slat:', raw_polesDF.SLat[line[i]], ', slon:', raw_polesDF.SLon[line[i]],\
                       ', age lower:', raw_polesDF.AgeLower[line[i]], ', age upper:', raw_polesDF.AgeUpper[line[i]]
    
 
# Create new GPlates Feature Collection
vpgFeatureCollection = pgp.FeatureCollection()
    
# Create new GPlates feature 'VirtualGeomagneticPole'.
# Pole lat, pole lon, pole name, and reconstruction plate ID added within PointOnSphere method.
# Inc, Dec, A95, Age and Sample site lat/lon values to added within 'other_properties' method.

for j in xrange(0, count):

    vgpFeature = pgp.Feature.create_reconstructable_feature(
                 pgp.FeatureType.create_gpml('VirtualGeomagneticPole'),
                 pgp.PointOnSphere([np.float(poleSiteLat[j]), np.float(poleSiteLon[j])]),
                 name = poleName[j],
                 reconstruction_plate_id = int(plateID[j]),
                 valid_time=(np.float(poleAgeUpperLimit[j]), np.float(poleAgeLowerLimit[j])),
                 other_properties = [(pgp.PropertyName.create_gpml('poleA95'), pgp.XsDouble(poleA95[j])),
                                     (pgp.PropertyName.create_gpml('averageAge'), pgp.XsDouble(poleNominalAge[j])),
                                     (pgp.PropertyName.create_gpml('averageSampleSitePosition'),
                                      pgp.GmlPoint(pgp.PointOnSphere([np.float(poleSiteLat[j]), 
                                                                      np.float(poleSiteLon[j])])))])

    # Add newly created feature to existing Feature Collection
    vpgFeatureCollection.add(vgpFeature)
    
    
print " "
print str(len(poleLat)) + " VGP features created"

 
177 VGP features created


In [17]:
# Generate GPML output file
gpmlOutputFile = "Mongolia_PmagPole_VGPs.gpml"

# Check for existing output directory and create it if not found
if not os.path.exists("output"):
    os.makedirs("output")

# Check for existing output file with same name and remove if found
if os.path.isfile("output/" + str(gpmlOutputFile)):
    os.remove("output/" + str(gpmlOutputFile))

# Check to make sure vgpFeatureCollection (feature collection) is not empty before writing to file
if len(vpgFeatureCollection) != 0:
    outputFeatureCollection = pgp.FeatureCollectionFileFormatRegistry()
    outputFeatureCollection.write(vpgFeatureCollection, "output/" + str(gpmlOutputFile))
    

# Check if new file was created and confirm export
if os.path.isfile("output/" + str(gpmlOutputFile)):
    print "Palaeomagnetic pole data successfully exported in GPML format"

Palaeomagnetic pole data successfully exported in GPML format
